In [1]:
#os.chdir('/home/lichi/jupiter/imaginarium')
import pandas as pd
import numpy as np
import torch
import pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%pylab inline

from lib.tools import cyclical_lr, learning_pipeline
from lib.data_preprocessing import pca_transfrom


Populating the interactive namespace from numpy and matplotlib


#### Загружаем данные

In [2]:
with open('preprocessed_data/words_coco_vectors.pickle', 'rb') as f:
    words_vec = pickle.load(f, encoding='latin1')
with open('preprocessed_data/image_coco_vectors.pickle', 'rb') as f:
    img_vec = pickle.load( f, encoding='latin1')

In [3]:
words_vec = pd.read_csv('data/words/coco/4000_coco.csv')
del words_vec['Unnamed: 0']
words_vec = np.array(words_vec)
img_vec = pd.read_csv('data/images/coco/coco_img.csv')
del img_vec['Unnamed: 0']
img_vec = np.array(img_vec)

#### Бьем на трейн и тест

### Делаем модель

In [5]:
# Датасет - подготавливаем наборы из 10 картинок , выбираем из них случайно одну
params = {
    'num_samples': 300,
    'num_cards': 5,
    'num_words': 998,
    'len_emb': 20,
    'len_dense': 20,
    'num_players':2,
    'gumbel': False,
    'comb_layer': False,
    'img_layer':True,
    'words_layer': False,
    'same_img_words_layer': True
}

config_train = {
    'num_batches': 800,
    'end_lr': 0.5,
    'factor': 10,
    'step_size': 300,
    'model_name': ''
}


### Учим модель

In [7]:
config_train['model_name'] = 'experiment_weights'
result = learning_pipeline(img_vec, words_vec, params, config_train)


Procent done 0 0/home/lichi/imaginarium/lib/model_classes.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights_words = self._softmax_words(logits)
/home/lichi/imaginarium/lib/model_classes.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._softmax_final(logits_final)
Procent done 0.18%          accuracy = 0.21


KeyboardInterrupt: 

### Проводим эксперименты

In [ ]:
result_list = []
list_len_layers = [20]
comb_list = [False, True]
same_img_words_layer_list = [False, True]
use_pca_before = [True, False]
words_layer_list = [False, True]
img_layer_list = [True, True]


In [ ]:

for len_layer in list_len_layers: 
    for comb_layer in comb_list:
        for same_img_words_layer in same_img_words_layer_list:
            for pca_use in use_pca_before:
                for words_layer in words_layer_list:
                    for img_layer in img_layer_list:
                        if (words_layer == False) & (img_layer == False):
                            pass
                        else:
                            config_train['model_name'] = ''
                            params_tmp = params
                            config_train_tmp = config_train

                            if pca_use:
                                params_tmp['len_emb'] = len_layer

                            params_tmp['len_dense'] = len_layer
                            params_tmp['words_layer'] = words_layer
                            params_tmp['img_layer'] = img_layer
                            params_tmp['comb_layer'] = comb_layer
                            params_tmp['same_img_words_layer'] = same_img_words_layer

                            config_train_tmp['model_name'] = \
                            'Model, len_layer={0}, comb_layer={1}, same_img_words_layer={2}, pca_use={3}, words_layer={4}, img_layer={5}'.format(
                                len_layer, comb_layer, same_img_words_layer, pca_use,words_layer,img_layer)
                            print(config_train_tmp['model_name'])
                            result_list.append(
                                learning_pipeline(img_vec, words_vec, params_tmp, config_train_tmp))


In [ ]:
same_img_words_layer

### Рисуем картинки

In [ ]:
folder = '2_players/'
title = 'Accuracy network after pca, depends on len_emb, num_players = {0}, num_cards = {2}'.format(
    params['num_players'], params['len_emb'], params['num_cards'])
for r,l in zip(result_list, list_len_layers):
    plt.plot(r['accuracy_list'],
             label = 'network, len embedding = {0}'.format(l))#r['config_network']['len_emb']))
    plt.plot([r['base_accuracy']]*1000, 
             label = 'pca baseline, len embedding = {0}'.format(l))#r['config_network']['len_emb']))



plt.ylabel('Accuracy')
plt.title(title)
plt.xlim((-50,1000))
plt.legend()
savefig('pictures/{0}/{1}.png'.format(folder, title))

!mkdir experiments_pickle
with open('experiments_pickle/{0}.pickle'.format(title), 'wb') as f:
    pickle.dump(result_list, f)

In [ ]:
r['base_accuracy']

### Шапка для картинок


In [ ]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Ubuntu'
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=10) 
plt.rc('ytick', labelsize=10)
# чтобы был русский шрифт
from matplotlib import rc
font = {'family': 'Verdana',
        'weight': 'normal'}
rc('font', **font)
